In [28]:
import lmdb
from os.path import join
import os
import caffe
import numpy as np
import hashlib
from collections import OrderedDict
import re

In [29]:
db_path_train = join('/home/aachenhang/dataset/cc/data', 'ExpoDB', 'Ratio0.4', 'train-count')
db_path_test = join('/home/aachenhang/dataset/cc/data', 'ExpoDB', 'Ratio0.4', 'test-count')

In [30]:
def is_image(im):
    return ('.jpg' in im) or ('.JPG' in im) or ('.PNG' in im) or ('.png' in im)

In [31]:
train_list = [img for img in os.listdir(join('/home/aachenhang/dataset/frames_Expo', 'train_frame')) if is_image(img)]
assert (len(train_list) != 0)

test_list = [img for img in os.listdir('/home/aachenhang/dataset/frames_Expo/test/') if is_image(img)]
assert (len(test_list) != 0)
#Shuffle the train set
img_hash = {hashlib.md5(img).hexdigest(): img for img in train_list}
img_hash = OrderedDict(sorted(img_hash.items()))
train_list = img_hash.values()

#Shuffle the test set
img_hash = {hashlib.md5(img).hexdigest(): img for img in test_list}
img_hash = OrderedDict(sorted(img_hash.items()))
test_list = img_hash.values()

In [32]:
train_cnt = np.array([])
minCount = 0
scale = 1
tol = 2

with open(join('/home/aachenhang/dataset/cc', 'data','ExpoDB', 'Ratio0.4','db.info')) as db_info:
    l = db_info.readlines()[1].split(',')  
    minCount = int(re.sub("[^0-9]", "", l[2]))
    scale = float(re.sub("[^0-9\.]", "", l[-1]))
print "minCount : ", minCount
print "scale : ", scale

minCount :  0
scale :  3.34


In [33]:
with lmdb.open(db_path_train, map_size=int(1e12)) as db_train:
    in_txn = db_train.begin(write=False)
    IDX_FMT = '{:0>%d' % 10 + 'd}'
    datum = caffe.proto.caffe_pb2.Datum()
    for i, img in enumerate(train_list):
        datum.ParseFromString(in_txn.get(IDX_FMT.format(i)))
        arr = caffe.io.datum_to_array(datum)
        pred = arr[:,0,0].argsort()[-1] + minCount
        pred = scale * pred
        label = int(re.sub("[^0-9]", "", img.split('n')[-1]))
        if(abs(pred - label) > tol):
            print "found error data!"
            print "index : ", i
            print "img name : ", img
            print "pred : ", pred
            print "label : ", label
            raise Exception("Found error data!")
    print "db_train pass"

db_train pass


In [34]:
with lmdb.open(db_path_test, map_size=int(1e12)) as db_test:
    in_txn = db_test.begin(write=False)
    IDX_FMT = '{:0>%d' % 10 + 'd}'
    datum = caffe.proto.caffe_pb2.Datum()
    for i, img in enumerate(test_list):
        datum.ParseFromString(in_txn.get(IDX_FMT.format(i)))
        arr = caffe.io.datum_to_array(datum)
        pred = arr[:,0,0].argsort()[-1] + minCount
        pred = scale * pred
        label = int(re.sub("[^0-9]", "", img.split('n')[-1]))
        if(abs(pred - label) > tol):
            print "found error data!"
            print "index : ", i
            print "img name : ", img
            print "pred : ", pred
            print "label : ", label
            raise Exception("Found error data!")
    print "db_test pass"
        

db_test pass
